In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('../')

In [6]:
from models import resnet_v1, resnet_v2
from utils import setup, viz

In [5]:
setup.set_seed()
cuda = setup.is_cuda()
device = setup.get_device()

[INFO] seed set 1
[INFO] Cuda Avaliable :  False
[INFO] device :  cpu


In [8]:
# model = resnet_v1.ResNet18
# model = resnet_v2.ResNet18
# net = model().to(device)
# viz.show_model_summary(net)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [54]:
test_data = torch.randn(1, 3, 32, 32)

In [10]:
x = nn.Conv2d(3, 64, 3, 1, 1, bias=False)

In [12]:
x(test_data).shape

torch.Size([1, 64, 28, 28])

In [79]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, 
            stride=stride, padding=stride, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3,
            stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_planes, self.expansion*planes,
                    kernel_size=1, stride=stride, padding=(stride - 1), bias=False
                ),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [80]:
def make_layer_mock(block, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    print("strides -- ", strides)
    global in_planes
    
    layers = []
    for stride in strides:
        layers.append(block(in_planes, planes, stride))
        print("in_planes, planes, stride -- ", in_planes, planes, stride)
        in_planes = planes * 1
    return nn.Sequential(*layers)

In [81]:
in_planes = 64
b1 = make_layer_mock(BasicBlock, planes=64, num_blocks=2, stride=1)
print("BLOCK 1 -- ")
print(b1)

b2 = make_layer_mock(BasicBlock, planes=128, num_blocks=2, stride=2)
print("BLOCK 2 -- ")
print(b2)

b3 = make_layer_mock(BasicBlock, planes=256, num_blocks=2, stride=2)
print("BLOCK 3 -- ")
print(b3)

b4 = make_layer_mock(BasicBlock, planes=512, num_blocks=2, stride=2)
print("BLOCK 4 -- ")
print(b4)

strides --  [1, 1]
in_planes, planes, stride --  64 64 1
in_planes, planes, stride --  64 64 1
BLOCK 1 -- 
Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )
)
strides --  [2, 1]
in_planes, planes, stride --

In [82]:
n = b1(torch.randn(1, 64, 32, 32)).shape
n

torch.Size([1, 64, 32, 32])

In [83]:
b2(torch.randn(n)).shape

torch.Size([1, 128, 17, 17])

In [85]:
b_1_2 = nn.Sequential(*[b1, b2])

In [86]:
b_1_2(torch.randn(1, 64, 32, 32)).shape

torch.Size([1, 128, 17, 17])